# BigFrames API Coverage Report

This notebook estimates BigFrame's coverage of Pandas APIs by:

1. Extracting the members from pandas, pandas.DataFrame, pandas.Index, pandas.Series
2. Checking if they are present in BigFrames
3. Searching them all on a dataset of 170k unique ipython notebooks queried from GitHub

TODO(bmil): add estimates based on Kaggle notebooks

In [1]:
import pandas as pd
import bigframes as bf
import inspect

token_map = {}
targets = [("pandas", pd, bf), ("dataframe", pd.DataFrame, bf.DataFrame), ("series", pd.Series, bf.Series), ("index", pd.Index, None)]
indexers = ['loc', 'iloc', 'iat', 'ix', 'at']
for name, pandas_obj, bigframes_obj in targets:
    for member in dir(pandas_obj):
        # skip private functions and properties
        if member[0] == "_" and member[1] != "_":
            continue

        if inspect.isfunction(getattr(pandas_obj, member)):
            # Function, match .member(
            token = f"\\.{member}\\("
            token_type = "function"
        elif member in indexers:
            # Indexer, match .indexer[
            token = f"\\.{member}\\["
            token_type = "indexer"
        else:
            # Property
            token = f"\\.{member}\\b"
            token_type = "property"

        is_in_bigframes = hasattr(bigframes_obj, member)

        # Special case: bigframes also implements some top level APIs on 'session'
        if name == "pandas":
            is_in_bigframes = is_in_bigframes or hasattr(bf.Session, member)

        if token not in token_map:
            token_map[token] = ([], token_type, is_in_bigframes)

        token_map[token][0].append(name)

header = ['pattern', 'token_type', 'is_pandas', 'is_dataframe', 'is_series', 'is_index', 'is_in_bigframes']
rows = [[
        k, v[1], 'pandas' in v[0], 'dataframe' in v[0], 'series' in v[0], 'index' in v[0], v[2]
                ] for k, v in token_map.items()]

# Wishlist: constructors for BigFrames...
pandas_df = pd.DataFrame(rows, columns=header)
pandas_df

/tmp/ipykernel_3446083/3523007326.py:14: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  if inspect.isfunction(getattr(pandas_obj, member)):
/tmp/ipykernel_3446083/3523007326.py:14: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  if inspect.isfunction(getattr(pandas_obj, member)):
/tmp/ipykernel_3446083/3523007326.py:14: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  if inspect.isfunction(getattr(pandas_obj, member)):


,pattern,token_type,is_pandas,is_dataframe,is_series,is_index,is_in_bigframes
0,\.ArrowDtype\b,property,True,False,False,False,False
1,\.BooleanDtype\b,property,True,False,False,False,False
2,\.Categorical\b,property,True,False,False,False,False
3,\.CategoricalDtype\b,property,True,False,False,False,False
4,\.CategoricalIndex\b,property,True,False,False,False,False
...,...,...,...,...,...,...,...
495,\.symmetric_difference\(,function,False,False,False,True,False
496,\.to_flat_index\(,function,False,False,False,True,False
497,\.to_native_types\(,function,False,False,False,True,False
498,\.to_series\(,function,False,False,False,True,False


This query will count the occurrence of the API patterns in a dump of 170,000 deduped IPython notebooks taken from the public GitHub dataset.

In [2]:
session = bf.connect()
df = session.read_pandas(pandas_df)

# Soon, we could do all this in BigFrames...... 🤞
# TODO: see how much we can rewrite
sql = f"""
WITH
  token_patterns AS ( {df.sql} ),
  kaggle_hit_counts AS (
    SELECT pattern, private_matches, public_matches, private_scanned, public_scanned
    FROM `bigframes-dev.coverage_report.kaggle_pandas_hit_counts`),
  github_notebooks AS (SELECT content FROM `bigframes-dev.coverage_report.github_notebooks`),
  github_hit_counts AS (
    SELECT
      token_patterns.pattern,
      COUNTIF(REGEXP_CONTAINS(github_notebooks.content, token_patterns.pattern)) AS matches,
      COUNT(*) AS scanned
    FROM
      token_patterns, github_notebooks
    WHERE CONTAINS_SUBSTR(github_notebooks.content, 'import pandas')
    GROUP BY token_patterns.pattern
    ORDER BY matches DESC
)
SELECT
  token_patterns.*,
  github_hit_counts.matches AS github_matches,
  github_hit_counts.scanned AS github_scanned,
  kaggle_hit_counts.private_matches AS kaggle_priv_matches,
  kaggle_hit_counts.private_scanned AS kaggle_priv_scanned,
  kaggle_hit_counts.public_matches AS kaggle_pub_matches,
  kaggle_hit_counts.public_scanned AS kaggle_pub_scanned
FROM token_patterns
LEFT JOIN kaggle_hit_counts ON token_patterns.pattern = kaggle_hit_counts.pattern
LEFT JOIN github_hit_counts ON token_patterns.pattern = github_hit_counts.pattern
"""

df = session.read_gbq(sql)

df

                pattern token_type  is_pandas  is_dataframe  is_series  \
0        \.ArrowDtype\b   property       True         False      False   
1      \.BooleanDtype\b   property       True         False      False   
2       \.Categorical\b   property       True         False      False   
3  \.CategoricalDtype\b   property       True         False      False   
4  \.CategoricalIndex\b   property       True         False      False   
5        \.DateOffset\b   property       True         False      False   
6     \.DatetimeIndex\b   property       True         False      False   
7   \.DatetimeTZDtype\b   property       True         False      False   
8         \.ExcelFile\b   property       True         False      False   
9       \.ExcelWriter\b   property       True         False      False   

   is_index  is_in_bigframes  github_matches  github_scanned  \
0     False            False               0           40337   
1     False            False               1           40

In [3]:
# TODO: rewrite everything in bigframes. Too much missing right now :(
df = df.to_pandas()
df

,pattern,token_type,is_pandas,is_dataframe,is_series,is_index,is_in_bigframes,github_matches,github_scanned,kaggle_priv_matches,kaggle_priv_scanned,kaggle_pub_matches,kaggle_pub_scanned
0,\.ArrowDtype\b,property,True,False,False,False,False,0,40337,<NA>,<NA>,<NA>,<NA>
1,\.BooleanDtype\b,property,True,False,False,False,False,1,40337,51,20398148,0,3429346
2,\.Categorical\b,property,True,False,False,False,False,233,40337,67152,20398148,17157,3429346
3,\.CategoricalDtype\b,property,True,False,False,False,False,25,40337,18254,20398148,1882,3429346
4,\.CategoricalIndex\b,property,True,False,False,False,False,3,40337,1903,20398148,255,3429346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,\.name\b,property,False,False,True,True,True,3146,40337,1157307,20398148,118453,3429346
496,\.__init__\b,property,False,False,False,True,True,733,40337,3317178,20398148,167504,3429346
497,\.__new__\(,function,False,False,False,True,True,17,40337,2816,20398148,49,3429346
498,\.__reduce__\(,function,False,False,False,True,True,0,40337,16,20398148,1,3429346


In [6]:
df["api"] = df["pattern"].str.replace('[\W(\\\\b)]', '')

# Clean out some properties that are also common python methods
# This is a limitation of the analysis
df = df[~df["api"].isin(["append", "tolist", "format", "items", "keys"])].copy()

/tmp/ipykernel_3446083/301973143.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["api"] = df["pattern"].str.replace('[\W(\\\\b)]', '')


In [7]:
# Weighted total API coverage
matches_stats = df[["github_matches", "kaggle_priv_matches", "kaggle_pub_matches"]]
covered = matches_stats[df["is_in_bigframes"]].sum(axis=0)
total = matches_stats.sum(axis=0)
weighted_percentage = covered * 100 / total
weighted_percentage["average"] = weighted_percentage.mean()
weighted_percentage

github_matches         36.001497
kaggle_priv_matches    37.238972
kaggle_pub_matches     37.199425
average                36.813298
dtype: float64

In [8]:
df["github_pct"] = df["github_matches"] * 100 / df["github_scanned"]
df["kaggle_priv_pct"] = df["kaggle_priv_matches"] * 100 / df["kaggle_priv_scanned"]
df["kaggle_pub_pct"] = df["kaggle_pub_matches"] * 100 / df["kaggle_pub_scanned"]
df["avg_pct"] = (df["github_pct"] + df["kaggle_priv_pct"] + df["kaggle_pub_pct"]) / 3

summary = df[["api", "avg_pct", "github_pct", "kaggle_priv_pct", "kaggle_pub_pct", "is_pandas", "is_dataframe", "is_series", "is_index", "is_in_bigframes"]]
summary = summary.sort_values("avg_pct", ascending=False)

print("The top missing APIs, and the rate at which they appear are:")
summary[summary["is_in_bigframes"] == False].head(50)

The top missing APIs, and the rate at which they appear are:


/tmp/ipykernel_3446083/4131997004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["github_pct"] = df["github_matches"] * 100 / df["github_scanned"]
/tmp/ipykernel_3446083/4131997004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["kaggle_priv_pct"] = df["kaggle_priv_matches"] * 100 / df["kaggle_priv_scanned"]
/tmp/ipykernel_3446083/4131997004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,api,avg_pct,github_pct,kaggle_priv_pct,kaggle_pub_pct,is_pandas,is_dataframe,is_series,is_index,is_in_bigframes
346,values,32.634977,28.254456,36.637184,33.013292,False,True,True,True,False
202,mean,29.681571,26.695094,35.745422,26.604198,False,True,True,False,False
213,plot,29.017144,41.019411,16.955951,29.076069,False,True,True,False,False
245,sum,27.423807,20.370875,32.203629,29.696916,False,True,True,False,False
324,astype,23.895698,15.055656,32.842354,23.789084,False,True,True,True,False
197,loc,23.863418,20.370875,28.866165,22.353213,False,True,True,False,False
351,array,21.421821,22.002132,26.226249,16.037081,False,False,True,True,False
50,array,21.077271,21.607953,25.807519,15.816339,True,False,False,False,False
182,iloc,19.73335,13.597937,22.6275,22.974614,False,True,True,False,False
135,apply,18.815486,15.945658,22.258545,18.242254,False,True,True,False,False
